In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

Pydantic Syntax
Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic.
They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.
In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

bar is not an integer. If it says it's an integer, it should throw up an error. Pydantic will do it

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

In [11]:
#foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com") #Expected to generate error

In [12]:
class Class(BaseModel):
    students: List[pUser]

Data can be nested

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

Use Pydantic to create OpenAI function format

In [16]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [50]:
from langchain_core.utils.function_calling import convert_to_openai_function

Converts Pydantic to JSON format required for OpenAI function. Easier to write it this way rather than writing a json file

In [20]:
# Convert Pydantic to JSON format required for OpenAI function
weather_function = convert_to_openai_function(WeatherSearch)

In [32]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

In [22]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [24]:
convert_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': "Usage docs: https://docs.pydantic.dev/2.8/concepts/models/ A base class for creating Pydantic models. Attributes:\n    __class_vars__: The names of classvars defined on the model.\n    __private_attributes__: Metadata about the private attributes of the model.\n    __signature__: The signature for instantiating the model.     __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.\n    __pydantic_core_schema__: The pydantic-core schema used to build the SchemaValidator and SchemaSerializer.\n    __pydantic_custom_init__: Whether the model has a custom `__init__` function.\n    __pydantic_decorators__: Metadata containing the decorators defined on the model.\n        This replaces `Model.__validators__` and `Model.__root_validators__` from Pydantic V1.\n    __pydantic_generic_metadata__: Metadata for generic models; contains data used for a similar purpose to\n        __args__, __origin__, __paramete

In [25]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [26]:
convert_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

Now, lets combine openai functions with langchain expression language

In [27]:
from langchain_openai import ChatOpenAI

In [28]:
model = ChatOpenAI()

In [29]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-fbc6f0b7-a138-4693-b0a1-b3e9299771e4-0', usage_metadata={'input_tokens': 61, 'output_tokens': 17, 'total_tokens': 78})

In [30]:
model_with_function = model.bind(functions=[weather_function])

In [31]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5b38f0c0-771c-48f9-8569-89b749e3accb-0', usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77})

In [33]:
#Forcing it to use a function
#We can force the model to use a function
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [34]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 70, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f1fe06f-f388-4ed9-9a1d-9d5432ec9e32-0', usage_metadata={'input_tokens': 70, 'output_tokens': 7, 'total_tokens': 77})

In [35]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 65, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-94958c8c-e432-469e-ae26-5ad25df62c7a-0', usage_metadata={'input_tokens': 65, 'output_tokens': 7, 'total_tokens': 72})

Using in a chain
We can use this model bound to function in a chain as we normally would

In [36]:
from langchain.prompts import ChatPromptTemplate

In [37]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [39]:
chain = prompt | model_with_function

In [41]:
chain.invoke({"input": "what is the weather in boston?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"BOS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 67, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b911f594-11dd-406d-9742-c68d68672150-0', usage_metadata={'input_tokens': 67, 'output_tokens': 17, 'total_tokens': 84})

Using multiple functions: 
Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [42]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [43]:
functions = [
    convert_to_openai_function(WeatherSearch),
    convert_to_openai_function(ArtistSearch),
]

In [44]:
model_with_functions = model.bind(functions=functions)

In [45]:
#asking about weather only calls weather function
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b824e6ee-e55e-49d3-84ec-d7f79fb38eb9-0', usage_metadata={'input_tokens': 93, 'output_tokens': 17, 'total_tokens': 110})

In [47]:
#asking about mushc only calls music function
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-a945d1c7-c117-464c-83c2-5b8b0a008faa-0', usage_metadata={'input_tokens': 95, 'output_tokens': 21, 'total_tokens': 116})

In [49]:
model_with_functions.invoke("can you give me the best song for boston weather")

AIMessage(content='Sure! To recommend the best song for Boston weather, I need to know the current weather in Boston. Let me check that for you.', additional_kwargs={'function_call': {'arguments': '{"airport_code":"BOS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ea8544dd-00af-4ac1-a7d6-c4ec74b3c9da-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})